Y-Maze Spontaneous Alternation Analysis

Files should be named as such: groupnumber_conditionandid_dayofrecording

e.g.
For the first group, the first sham mouse at day 2 should be named: g1_sham1_d02

Analyze videos with DeepLabCut, the points should be written like this and in this order (you can copy and paste this in the config file):
- nose
- ear_left
- ear_right
- tail_base
- y_top
- y_left
- y_right
- center_bottom
- center_left
- center_right

Write the path of the *.h5 files in the variable below, between the ""

In [ ]:
directory = r"path\to\directory"

Run the cell below to create a csv with added metrics for each recording and a collected csv for all the files in the folder (it's important to have, in the same folder as this script, the util.py file)

In [ ]:
from util import create_dataset, add_metrics, combine_rows

df = create_dataset(directory)
df  = add_metrics(df=df, directory=directory)
df = combine_rows(df)
df.to_csv("collected_dataset_metrics.csv", index=False)

(OPTIONAL) Script to visualize videos with prediction and arm

In [ ]:
# VIDEO FILE PATH
file_name = r"path\to\video"
# DATA PATH
data_file_name = r"path\to\corresponding\csv"

In [ ]:
import cv2
import platform
import pandas as pd
from util import overlay_pose_estimation

df = pd.read_csv(data_file_name)
# Video reproduction
camera = cv2.VideoCapture(file_name)
WINDOW_NAME = "img"
cv2.namedWindow(WINDOW_NAME)
cv2.startWindowThread()
length = int(camera.get(cv2.CAP_PROP_FRAME_COUNT))
frame_index = 1
while True:
    ret, img = camera.read()
    if ret:
        h, w, c = img.shape
        df_line = df.iloc[frame_index]

        comb_img = overlay_pose_estimation(img, df_line, w, h)
        cv2.putText(
            comb_img,
            str(df_line["arm"]),
            (10, 500),
            4,
            4,
            (255, 255, 255),
            2,
            cv2.LINE_AA,
        )

        cv2.imshow(WINDOW_NAME, comb_img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        frame_index += 1
    else:
        break

camera.release()
cv2.destroyAllWindows()

# Next line helps with an error with macOS not closing windows and freezing
if platform.system() == "Darwin":
    for i in range(1, 5):
        cv2.waitKey(1)
